# ANLY590 Lab8 RNN
## Yunxi Zhang yz721

In [1]:
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
from tensorflow.keras.models import Sequential


import numpy as np
import pandas as pd

In this example, we're going to train a [CharRNN](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) on a body of Shakespearian text. Ultimtely, this is an unsuperived learning task. But similar to our previous explorations in unsupervised DL, we will use an unlabeled dataset and create many samples of labeled data that we can use with our familiar supervised loss functions. The result will be a model that has learned the statistical properties of the input text, and can then be considered a "generative" model of language because we can use it to generate synthetic passages of Shakespeare.  

In [ ]:
drive.mount('/content/gdrive/')

In [2]:
file_path = "shakespeare.txt"

with open(file_path,"r") as f:
  text = f.read()

We've loaded our Shakespeare text, let's take a look at a random snippet.

In [3]:
print(text[31600:32000])

 lies i' the second chamber?
  LADY MACBETH. Donalbain.
  MACBETH. This is a sorry sight.           [Looks on his hands.
  LADY MACBETH. A foolish thought, to say a sorry sight.
  MACBETH. There's one did laugh in 's sleep, and one cried,
      "Murther!"
    That they did wake each other. I stood and heard them,
    But they did say their prayers and address'd them
    Again to sleep.
  LADY MACB


We need to convert our text into numeric arrays, the next several blocks accomplish this.

First, we'll create a mapping between characters and their numeric index. We'll also create the reverse mapping, which is useful.

In [4]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 75


Next, we'll create a training set of sub-sequences. Remember, we're trying to train a model to be able to predict the next chracter if it is given several characters of a subsequence. So we will create training pairs where each X is a fixed-length subsequences and each Y is the corresponding next letter in the text.

In [5]:
maxlen = 40
step = 3
sub_sequences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sub_sequences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sub_sequences))

nb sequences: 38700


In [6]:
k=300
print("(Sequence):\n" + sub_sequences[k])
print("\n(Target Character): \n" + next_chars[k])

(Sequence):
 and other Apparitions
  Lords, Gentleme

(Target Character): 
n


Next we'll create one-hot vectors for our sub-sequences. The tensor we create here will be shaped as (num_sequences x sequence_length x alphabet_size).

In [7]:
X = np.zeros((len(sub_sequences), maxlen, len(chars)), dtype=np.uint8 )
Y = np.zeros((len(sub_sequences), len(chars)), dtype=np.uint8)
for i, seq in enumerate(sub_sequences):
    for t, char in enumerate(seq):
        X[i, t, char_indices[char]] = 1
        Y[i, char_indices[next_chars[i]]] = 1

In [8]:
X[0,0,:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=uint8)

In [9]:
Y[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

Our RNN model will be quite simple.

In [10]:
char_rnn = Sequential()
char_rnn.add(tfkl.LSTM(128, input_shape=(maxlen, len(chars))))
char_rnn.add(tfkl.Dense(len(chars),activation="softmax"))

In [11]:
char_rnn.compile(loss='categorical_crossentropy', optimizer=tfk.optimizers.RMSprop(lr=0.01))

In [12]:
char_rnn.fit(X,Y, epochs=20, batch_size=1024)

Epoch 1/20
38/38 [==============================] - 8s 222ms/step - loss: 3.2415
Epoch 2/20
38/38 [==============================] - 9s 248ms/step - loss: 2.8969
Epoch 3/20
38/38 [==============================] - 10s 253ms/step - loss: 2.4236
Epoch 4/20
38/38 [==============================] - 9s 249ms/step - loss: 2.1411
Epoch 5/20
38/38 [==============================] - 10s 250ms/step - loss: 1.9729
Epoch 6/20
38/38 [==============================] - 10s 253ms/step - loss: 1.8488
Epoch 7/20
38/38 [==============================] - 10s 252ms/step - loss: 1.7601
Epoch 8/20
38/38 [==============================] - 10s 251ms/step - loss: 1.6822
Epoch 9/20
38/38 [==============================] - 10s 254ms/step - loss: 1.6185
Epoch 10/20
38/38 [==============================] - 10s 253ms/step - loss: 1.5556
Epoch 11/20
38/38 [==============================] - 10s 258ms/step - loss: 1.6484
Epoch 12/20
38/38 [==============================] - 10s 256ms/step - loss: 1.5892
Epoch 13/20
38/3

Once we have a trained model, we can simulate new text by making predictions about the next character and then drawing characters in proportion to the predicted probabilities. And then simple repeat that process over and over, each time drawing the next character.

In [13]:
def draw_char(probs):
    probs = np.asarray(probs).astype('float64')
    if sum(probs) != 1.0:
      probs = probs / np.sum(probs)
    draw = np.random.choice(range(len(probs)) , p=probs)
    return draw

def sample_text(model, sample_length=100):
    start = np.random.randint(0, len(text) - maxlen - 1)
    sequence = text[start: start + maxlen]
  
    x_preds = np.zeros((sample_length, maxlen, len(chars)))
    for i in range(sample_length):
        for t, char in enumerate(sequence[-maxlen:]):
            x_preds[i, t, char_indices[char]] = 1.

        preds = model.predict(np.expand_dims(x_preds[i,:,:], axis=0), verbose=0)[0]
        next_index = draw_char(preds)
        next_char = indices_char[next_index]

        sequence += next_char
    return sequence

In [14]:
sim = sample_text(char_rnn,sample_length=500) 

In [15]:
print(sim)

g the knowledge of the broil
    As thou the using, my kere you, tho gaif,
    Of how, a hos and hall and lisk befort all best not,
    But goight sught as mestre's tho ghors.
  DOCTOR. Shill a not thee hard covillt chill'd natur
    That where thet Doda Sunne.
  LADY MACBETH. Their great speek ho whe cable with the nath,
    Iw maring ESSSONG SEAN. Alay Fhear revon, the ofleftick out. Douss, mine,
    The ear for it hall hinds.
  GENNOX, ATtIIRT! As Dullove nothish
    The bleass noth but mud Dun" wool not, f'll strong?
  MAITHind si


Notice that we can do pretty well to learn the typical statistical patterns of this text and then simulate new text that appears to be very similar to legitimate Shakespeare. 

But just a caution - we can also do pretty well with a much simpler method (Markov model): http://nbviewer.jupyter.org/gist/yoavg/d76121dfde2618422139

So the lesson is to try something simple before jumping right in to deep learning.

## Exercise

In this example, we're going to use an RNN for sequence classification. The task we'll set up is to generate a training set of randomized strings, and train our model to detect whether a string contains any vowels.

First, we'll create a training dataset of short randomized character sequences and the corresponding label of whether or not they contain at least one vowel.

In [16]:
import string

In [17]:
def contains_vowels(sequence):
  vowels = ["a", "e", "i", "o", "u"]
  return any([vowel in list(sequence) for vowel in vowels])

In [18]:
contains_vowels("gradient")

True

In [19]:
sequences = []
labels = []
for i in range(1000):
 char_list = np.random.choice( list(string.ascii_lowercase), size = 5, replace=True)
 seq = "".join(char_list)
 sequences.append(seq)
 labels.append(int(contains_vowels(seq)))

In [20]:
df = pd.DataFrame({"sequence": sequences, "label":labels})

In [21]:
df.head()

,sequence,label
0,jyhnp,0
1,ggfum,1
2,cougy,1
3,fcboq,1
4,agdwi,1


Next, set up and train an RNN (of any type) to solve this task. What preprocessing will you need to do first on the raw data in order to prepare it for the network?

In [22]:
# Data Preprocessing
# your code here
chars1 = list(string.ascii_lowercase)
char_indices1 = dict((c, i) for i, c in enumerate(chars1))
indices_char1 = dict((i, c) for i, c in enumerate(chars1))

print('total chars1:',len(chars1))

total chars1: 26


In [25]:
maxlen1=5

X = np.zeros((len(sequences),maxlen1,len(chars1)), dtype=np.uint8)
Y = np.array(labels)

for i, seq in enumerate(sequences):
  for t,char in enumerate(seq):
        X[i,t,char_indices1[char]] = 1

In [26]:
X[0,0,:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0], dtype=uint8)

In [27]:
Y[0]

0

In [33]:
# Model setup and training
# your code here
char_rnn1 = Sequential()
char_rnn1.add(tfkl.LSTM(128, input_shape=(maxlen1,len(chars1))))
char_rnn1.add(tfkl.Dense(1,activation="sigmoid"))
char_rnn1.compile(loss='binary_crossentropy', optimizer=tfk.optimizers.RMSprop(lr=0.01),metrics=['accuracy'])
char_rnn1.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 128)               79360     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 79,489
Trainable params: 79,489
Non-trainable params: 0
_________________________________________________________________


In [46]:
char_rnn1.fit(X,Y,epochs=20,steps_per_epoch=10,batch_size=100)

Epoch 1/20
10/10 [==============================] - 0s 7ms/step - loss: 2.1184e-09 - accuracy: 1.0000
Epoch 2/20
10/10 [==============================] - 0s 6ms/step - loss: 2.1371e-09 - accuracy: 1.0000
Epoch 3/20
10/10 [==============================] - 0s 5ms/step - loss: 2.1161e-09 - accuracy: 1.0000
Epoch 4/20
10/10 [==============================] - 0s 5ms/step - loss: 2.0687e-09 - accuracy: 1.0000
Epoch 5/20
10/10 [==============================] - 0s 5ms/step - loss: 2.0699e-09 - accuracy: 1.0000
Epoch 6/20
10/10 [==============================] - 0s 5ms/step - loss: 2.1215e-09 - accuracy: 1.0000
Epoch 7/20
10/10 [==============================] - 0s 5ms/step - loss: 2.1297e-09 - accuracy: 1.0000
Epoch 8/20
10/10 [==============================] - 0s 5ms/step - loss: 2.1060e-09 - accuracy: 1.0000
Epoch 9/20
10/10 [==============================] - 0s 5ms/step - loss: 2.0154e-09 - accuracy: 1.0000
Epoch 10/20
10/10 [==============================] - 0s 5ms/step - loss: 2.1521e-0